In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Load the models and scaler
logistic_regression_model = joblib.load('logistic_regression_model.pkl')
decision_tree_model = joblib.load('DecisionTreeClassifier_model.pkl')
svm_model = joblib.load('SVM.pkl')
random_forest_model = joblib.load('random_forest_model.pkl')
scaler = joblib.load('scaler.pkl')

# Define the main application class
class PredictionApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Prediction App")
        self.geometry("400x700")
        self.frames = {}

        # Create and show the first frame
        self.show_frame(InputFrame)

    def show_frame(self, container):
        frame = container(self)
        self.frames[container] = frame
        frame.grid(row=0, column=0, sticky="nsew")
        frame.tkraise()

class InputFrame(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent

        # Define input fields
        self.inputs = {}

        # Campaign Type
        tk.Label(self, text="Campaign Type").grid(row=0, column=0, padx=10, pady=5)
        self.campaign_type = ttk.Combobox(self, values=["Yes", "No"])
        self.campaign_type.grid(row=0, column=1, padx=10, pady=5)
        self.inputs["Campaign Type"] = self.campaign_type

        # Marital Status
        tk.Label(self, text="Marital Status").grid(row=1, column=0, padx=10, pady=5)
        self.marital_status = ttk.Combobox(self, values=["Married", "Single"])
        self.marital_status.grid(row=1, column=1, padx=10, pady=5)
        self.inputs["Marital Status"] = self.marital_status

        # Rented
        tk.Label(self, text="Rented").grid(row=2, column=0, padx=10, pady=5)
        self.rented = ttk.Combobox(self, values=["True", "False"])
        self.rented.grid(row=2, column=1, padx=10, pady=5)
        self.inputs["Rented"] = self.rented

        # Family Size
        tk.Label(self, text="Family Size").grid(row=3, column=0, padx=10, pady=5)
        self.family_size = tk.Spinbox(self, from_=1, to=6)
        self.family_size.grid(row=3, column=1, padx=10, pady=5)
        self.inputs["Family Size"] = self.family_size

        # Selling Price
        tk.Label(self, text="Selling Price").grid(row=4, column=0, padx=10, pady=5)
        self.selling_price = tk.Entry(self)
        self.selling_price.grid(row=4, column=1, padx=10, pady=5)
        self.inputs["Selling Price"] = self.selling_price

        # Age Range
        tk.Label(self, text="Age Range").grid(row=5, column=0, padx=10, pady=5)
        self.age_range = ttk.Combobox(self, values=["18-25", "26-35", "36-45", "46-55", "56+"])
        self.age_range.grid(row=5, column=1, padx=10, pady=5)
        self.inputs["Age Range"] = self.age_range

        # No of Children
        tk.Label(self, text="No of Children").grid(row=6, column=0, padx=10, pady=5)
        self.no_of_children = tk.Spinbox(self, from_=1, to=5)
        self.no_of_children.grid(row=6, column=1, padx=10, pady=5)
        self.inputs["No of Children"] = self.no_of_children

        # Income Bracket
        tk.Label(self, text="Income Bracket").grid(row=7, column=0, padx=10, pady=5)
        self.income_bracket = tk.Entry(self)
        self.income_bracket.grid(row=7, column=1, padx=10, pady=5)
        self.inputs["Income Bracket"] = self.income_bracket

        # Quantity
        tk.Label(self, text="Quantity").grid(row=8, column=0, padx=10, pady=5)
        self.quantity = tk.Spinbox(self, from_=1, to=10)
        self.quantity.grid(row=8, column=1, padx=10, pady=5)
        self.inputs["Quantity"] = self.quantity

        # Brand Type
        tk.Label(self, text="Brand Type").grid(row=9, column=0, padx=10, pady=5)
        self.brand_type = ttk.Combobox(self, values=["Local", "Established"])
        self.brand_type.grid(row=9, column=1, padx=10, pady=5)
        self.inputs["Brand Type"] = self.brand_type

        # Category
        tk.Label(self, text="Category").grid(row=10, column=0, padx=10, pady=5)
        self.category = ttk.Combobox(self, values=["Electronics", "Clothing", "Home", "Beauty", "Sports"])
        self.category.grid(row=10, column=1, padx=10, pady=5)
        self.inputs["Category"] = self.category

        # Prod Income
        tk.Label(self, text="Prod Income").grid(row=11, column=0, padx=10, pady=5)
        self.prod_income = tk.Entry(self)
        self.prod_income.grid(row=11, column=1, padx=10, pady=5)
        self.inputs["Prod Income"] = self.prod_income

        # Next button
        next_button = tk.Button(self, text="Next", command=self.go_to_model_selection)
        next_button.grid(row=12, columnspan=2, pady=20)

    def go_to_model_selection(self):
        try:
            # Validate and store inputs
            self.feature_values = []
            for key, entry in self.inputs.items():
                value = entry.get()
                if not value:
                    messagebox.showerror("Input Error", f"Please enter a value for {key}")
                    return
                
                # Convert values based on input type
                if key == "Campaign Type":
                    value = 1 if value == "Yes" else 0
                elif key == "Marital Status":
                    value = 1 if value == "Married" else 0
                elif key == "Rented":
                    value = 1 if value == "True" else 0
                elif key == "Age Range":
                    value = ["18-25", "26-35", "36-45", "46-55", "56+"].index(value) + 1
                elif key == "Brand Type":
                    value = 1 if value == "Established" else 0
                elif key == "Category":
                    value = ["Electronics", "Clothing", "Home", "Beauty", "Sports"].index(value) + 1

                self.feature_values.append(float(value))
            self.parent.show_frame(ModelSelectionFrame)
        except ValueError:
            messagebox.showerror("Input Error", "Please enter numeric values for all fields")

class ModelSelectionFrame(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        tk.Label(self, text="Select Model").grid(row=0, column=0, padx=10, pady=10)
        self.model_selection = ttk.Combobox(self, values=[
            "Logistic Regression", "Decision Tree", "SVM", "Random Forest"
        ])
        self.model_selection.grid(row=0, column=1, padx=10, pady=10)
        self.model_selection.current(0)

        # Predict button
        predict_button = tk.Button(self, text="Predict", command=self.predict)
        predict_button.grid(row=1, columnspan=2, pady=20)

    def predict(self):
        # Get input values from previous frame
        feature_values = self.parent.frames[InputFrame].feature_values
        # Scale the input
        scaled_features = scaler.transform([feature_values])

        # Select the model
        selected_model = self.model_selection.get()
        if selected_model == "Logistic Regression":
            model = logistic_regression_model
        elif selected_model == "Decision Tree":
            model = decision_tree_model
        elif selected_model == "SVM":
            model = svm_model
        elif selected_model == "Random Forest":
            model = random_forest_model
        else:
            messagebox.showerror("Model Error", "Please select a valid model")
            return

        # Make prediction
        prediction = model.predict(scaled_features)
        prediction_proba = model.predict_proba(scaled_features)

        # Show the prediction result
        if prediction[0] == 1:
            result_text = "Yes will buy the coupon"
        else:
            result_text = "No will not buy the coupon"
        
        messagebox.showinfo("Prediction Result", f"Prediction: {result_text}\nProbability: {prediction_proba[0][1]:.2f}")

if __name__ == "__main__":
    app = PredictionApp()
    app.mainloop()


/home/nitin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/nitin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/nitin/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
